# Chicago Data TWAS
## Motivation
Before actual analysis of our own data, the pipeline shall first be used to try to reproduce the work that are done previously. So that the result can be compared.

## Data set
The data set is the gene expresion data from African population got from chicago university. It contains the expression profile of 50 genes from 89 individuals. The gene expression is processed by STAR and recorded as RPKM. 

## Tools
The tools used in this analysis is the "SOS_weight_cpt_template" pipeline, with the options outlined as followed. Developed on the fusion foundation, this pipeline takes in gene expression data ans SNP to compute the weight for the follow ups Correlation analysis. association testing are then conducted by the standard Fusion scripts

In [176]:
sos run SOS_weight_cpt_template.ipynb -h

usage: sos run SOS_weight_cpt_template.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  STEP

Global Workflow Options:
  --GCTA '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/gcta_1.93.2beta_mac/gcta64'
                        !/bin/sh MAKE SURE FUSION.compute_weights.R IS IN YOUR
                        PATH FILL IN THESE PATHS For mac user, the mac version
                        of GCTA shall be downloaded saperately, the one came
                        with the Fusion package will not work.
  --PLINK '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/plink_mac_20200616/plink'
  --GEMMA '/Users/haosun/Documents/WG_Reasea

## Test pipeline with example datas
The sos pipeline were tested using the provided working example, exhibiting similar behavior as described in the documents.

sos run SOS_weight_cpt_template.ipynb \
  --GCTA "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/gcta_1.93.2beta_mac/gcta64" \
  --PLINK `which plink` \
  --GEMMA `which gemma` \
  --Rscp  "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/FUSION.compute_weights.mod.R" \
  --Datafile "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/Testing/Data/GD462.hsq_succ.test.txt" \
  --wd  "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/SOS" \
  --OUT_DIR "./" \
  --LDREF  "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF" \
  --PRE_GENO "1000G.EUR"

# Data wrangling
Before conducting analysis on the chicago data, the following issues needed to be fixed via the data wrangling steps:
1. The sample ID is not equal to the bfile ID
Note: Please disregard the warning message that

In [5]:
##Ad hoc Data wrangling for the chicago data
library("dplyr")
to_fix = readr::read_delim("/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt",delim = "\t")
colnames(to_fix)[5:ncol(to_fix)] = paste("NA",colnames(to_fix)[5:ncol(to_fix)])%>%gsub(" ","",.)
to_fix%>%readr::write_tsv( path = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/fastqtl_qqnorm_RNAseqGeuvadis_phase2_fixed.txt", na = "NA", append = FALSE, col_names = TRUE, quote_escape = "double")
colnames(to_fix)

Parsed with column specification:
cols(
  .default = col_double(),
  `#Chr` = col_character(),
  ID = col_character()
)

See spec(...) for full column specifications.

Warning message:
“1 parsing failure.
row col   expected     actual                                                                                                                                      file
 49  -- 89 columns 74 columns '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt'
”


[1] "#Chr"    "start"   "end"     "ID"      "NA18486" "NA18487" "NA18488"
 [8] "NA18489" "NA18498" "NA18499" "NA18502" "NA18505" "NA18508" "NA18510"
[15] "NA18511" "NA18517" "NA18519" "NA18520" "NA18858" "NA18861" "NA18867"
[22] "NA18868" "NA18870" "NA18873" "NA18907" "NA18909" "NA18910" "NA18912"
[29] "NA18916" "NA18917" "NA18923" "NA18933" "NA18934" "NA19093" "NA19095"
[36] "NA19096" "NA19098" "NA19099" "NA19102" "NA19107" "NA19108" "NA19113"
[43] "NA19114" "NA19116" "NA19117" "NA19118" "NA19119" "NA19121" "NA19129"
[50] "NA19130" "NA19131" "NA19137" "NA19138" "NA19141" "NA19143" "NA19144"
[57] "NA19146" "NA19147" "NA19149" "NA19150" "NA19152" "NA19153" "NA19159"
[64] "NA19171" "NA19172" "NA19175" "NA19184" "NA19185" "NA19189" "NA19190"
[71] "NA19197" "NA19198" "NA19200" "NA19201" "NA19204" "NA19206" "NA19207"
[78] "NA19209" "NA19210" "NA19213" "NA19222" "NA19223" "NA19225" "NA19235"
[85] "NA19236" "NA19247" "NA19248" "NA19256" "NA19257"

## SVA correction (In progress)
Hidden factor is addressed for each genes using the sva method. Current obstacle of implementing this section is to create a model matrix with both the gene expression data and the snp information in the same matrixs for every genes.

In [1]:

## Test pipeline with chicago data
sos_pipeline="../Workflow/SOS_weight_cpt_template.ipynb"
sos run ../Workflow/SOS_weight_cpt_template.ipynb \
  --GCTA "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/gcta_1.93.2beta_mac/gcta64" \
  --PLINK `which plink` \
  --GEMMA `which gemma` \
  --Rscp  "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/FUSION.compute_weights.mod.R" \
  --Datafile "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/fastqtl_qqnorm_RNAseqGeuvadis_phase2_fixed.txt" \
  --wd  "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/working" \
  --LDREF  "/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/genotype_plink_bfiles" \
  --PRE_GENO "YRI.120samples.hg19" \
  --chrm 1 \
  --gname 4 \
  --start 2 \
  --end 3 \
  --window 1000


INFO: Running STEP_1: Make folder structure for the pipeline
INFO: STEP_1 (index=0) is ignored due to saved signature
INFO: STEP_1 (index=1) is ignored due to saved signature
INFO: STEP_1 (index=2) is ignored due to saved signature
INFO: STEP_1 (index=3) is ignored due to saved signature
INFO: STEP_1 (index=4) is ignored due to saved signature
INFO: STEP_1 (index=5) is ignored due to saved signature
INFO: STEP_1 (index=6) is ignored due to saved signature
INFO: STEP_1 (index=7) is ignored due to saved signature
INFO: STEP_1 (index=8) is ignored due to saved signature
INFO: STEP_1 (index=9) is ignored due to saved signature
INFO: STEP_1 (index=10) is ignored due to saved signature
INFO: STEP_1 (index=11) is ignored due to saved signature
INFO: STEP_1 (index=12) is ignored due to saved signature
INFO: STEP_1 (index=13) is ignored due to saved signature
INFO: STEP_1 (index=14) is ignored due to saved signature
INFO: STEP_1 (index=15) is ignored due to saved signature
INFO: STEP_1 (index=1

## Result
The above scripts was ran 3 times before the desired output was generated.

When scanning 1000bp before and after the gene region. Out of 50 genes, 10 of them passed the heritability test and can be passed down toward the followup analysis.

When scanning 500000bp before and after the gene starting site, out of 50 genes, 4 of them passed the heritability test and can be passed down toward the followup analysis.

In [ ]:
## Potential graphing with the hsq distribution

# Association testing
Association testing shall be done using the following scripts

In [13]:
Rscr2=""
sumstats=""
weights=""
weights_di=""
LDREF=""
PRE_GENO=""
out=""
## 
Rscript $Rscr2 \
--sumstats $sumstats \
--weights $weights \
--weights_dir $weights_dir \
--ref_ld_chr $LDREF/PRE_GENO \
--chr 22 \
--out $out

INFO: Running example :

ERROR: [example]: [example]: Output target $a.txt does not exist after the completion of step example
